In [7]:
import numpy as np

In **Gaussian elimination**, the goal is to transform a matrix $A$ into an **upper triangular matrix** $U$ by applying a series of **elementary row operations**. These operations can be represented as **elementary matrices**, and when we apply them to $A$, we get:

$$
E_k \cdots E_2 E_1 A = U
$$

This can be rearranged to:

$$
A = E_1^{-1} E_2^{-1} \cdots E_k^{-1} U
$$

Now, here's where the **LU decomposition** comes in. Instead of explicitly storing all the elimination steps as matrices, we combine their inverses (since we applied them to $A$) into one matrix:

$$
A = LU
$$

Where:

* $L = E_1^{-1} E_2^{-1} \cdots E_k^{-1}$ (the product of **inverse** of elimination matrices in **reverse order**)
* $U$ is the final upper triangular matrix from Gaussian elimination.

---

### Why this is beautiful:

If there are **no row exchanges (pivoting)**, then the multipliers used to eliminate entries below the diagonal go **directly** into the $L$ matrix. This makes $L$ a **lower triangular matrix with ones on the diagonal**, and the entries below the diagonal are just the elimination multipliers.

---

### Example:

Let’s take a simple matrix:

$$
A = \begin{bmatrix}
2 & 3 \\
4 & 7
\end{bmatrix}
$$

We perform Gaussian elimination to make it upper triangular:

* First, eliminate the (2,1) entry. The multiplier is $m = \frac{4}{2} = 2$.
* Row 2 becomes: $R_2 - 2 \cdot R_1 \Rightarrow [4, 7] - 2 \cdot [2, 3] = [0, 1]$

So the upper triangular matrix is:

$$
U = \begin{bmatrix}
2 & 3 \\
0 & 1
\end{bmatrix}
$$

The elimination matrix for this step is:

$$
E = \begin{bmatrix}
1 & 0 \\
-2 & 1
\end{bmatrix}
\quad \text{and its inverse is} \quad
E^{-1} = \begin{bmatrix}
1 & 0 \\
2 & 1
\end{bmatrix}
$$

So:

$$
A = E^{-1} U = LU
$$

Where:

$$
L = \begin{bmatrix}
1 & 0 \\
2 & 1
\end{bmatrix}, \quad
U = \begin{bmatrix}
2 & 3 \\
0 & 1
\end{bmatrix}
$$

Thus, we’ve decomposed $A$ into $LU$ form, and the entries of $L$ are the elimination multipliers (2 in this case) directly inserted into the lower part of the triangle.

---

### Summary:

* Gaussian elimination gets us from $A$ to $U$.
* LU decomposition rewrites $A$ as a product $LU$, encoding the elimination process.
* The matrix $L$ is made of inverse elimination matrices, which is elegant because it captures the whole elimination process compactly.
* No pivoting? Then $L$ neatly stores the multipliers without modification.


In [4]:
import numpy as np

def lu_decomposition_no_pivoting(A):
    """
    Perform LU decomposition of a 2x2 matrix A without pivoting.
    Assumes A is square and can be decomposed without row exchanges.
    """
    n = A.shape[0]
    L = np.eye(n)
    U = A.copy().astype(float)

    for i in range(n):
        for j in range(i+1, n):
            multiplier = U[j, i] / U[i, i]
            L[j, i] = multiplier
            U[j, :] = U[j, :] - multiplier * U[i, :]
    
    return L, U


In [5]:
# Example matrix from earlier:
A = np.array([[2, 3],
              [4, 7]], dtype=float)

L, U = lu_decomposition_no_pivoting(A)

print("Matrix A:")
print(A)
print("\nLower Triangular Matrix L:")
print(L)
print("\nUpper Triangular Matrix U:")
print(U)

# Verify A = LU
reconstructed_A = L @ U
print("\nReconstructed A from L and U:")
print(reconstructed_A)


Matrix A:
[[2. 3.]
 [4. 7.]]

Lower Triangular Matrix L:
[[1. 0.]
 [2. 1.]]

Upper Triangular Matrix U:
[[2. 3.]
 [0. 1.]]

Reconstructed A from L and U:
[[2. 3.]
 [4. 7.]]


In [6]:
B = np.array([[2, 3, 5],
              [4, 7, 6],
              [3, 5, 6]], dtype=float)

L, U = lu_decomposition_no_pivoting(B)

print("Matrix B:")
print(B)
print("\nLower Triangular Matrix L:")
print(L)
print("\nUpper Triangular Matrix U:")
print(U)

# Verify A = LU
reconstructed_B = L @ U
print("\nReconstructed B from L and U:")
print(reconstructed_B)

Matrix B:
[[2. 3. 5.]
 [4. 7. 6.]
 [3. 5. 6.]]

Lower Triangular Matrix L:
[[1.  0.  0. ]
 [2.  1.  0. ]
 [1.5 0.5 1. ]]

Upper Triangular Matrix U:
[[ 2.   3.   5. ]
 [ 0.   1.  -4. ]
 [ 0.   0.   0.5]]

Reconstructed B from L and U:
[[2. 3. 5.]
 [4. 7. 6.]
 [3. 5. 6.]]


To compare the **operation cost** of getting from matrix $A$ to $U$ (upper triangular) using two different methods — **Gaussian elimination (EA = U)** and **LU decomposition (A = LU)** — we need to look at the number of **floating point operations (flops)** involved.

---

## 🔧 Method 1: **Gaussian Elimination (EA = U)**

This method applies a series of **elementary row operations** to matrix $A$, transforming it directly to $U$. These operations can be represented as matrix multiplications:

$$
E_k \cdots E_2 E_1 A = U
$$

Each $E_i$ corresponds to eliminating one entry below the pivot.

### 🧮 Operation Cost (Flops):

Let’s consider a general **$n \times n$** matrix $A$. The process:

* For each pivot row $i$ from 1 to $n-1$, you:

  * Eliminate entries below the pivot in column $i$.
  * For each of those entries (say, $n - i$ rows), update the rest of the row (from column $i$ to $n$).

So the total number of operations is:

$$
\sum_{i=1}^{n-1} \sum_{j=i+1}^{n} (n - i + 1) \approx \frac{n^3}{3}
$$

### ✅ Pros:

* Simple conceptually.
* Good for direct solving without storing extra matrices.

### ❌ Cons:

* Does not separate the elimination process from the upper triangular matrix.
* If needed again for multiple right-hand sides (in $Ax = b$), you'd have to redo elimination.

---

## 🔧 Method 2: **LU Decomposition (A = LU)**

Here, we **factor** matrix $A$ into a lower triangular matrix $L$ and an upper triangular matrix $U$, such that:

$$
A = LU
$$

This involves **storing the multipliers** (from Gaussian elimination) in matrix $L$, and keeping the result in $U$.

### 🧮 Operation Cost (Flops):

Surprisingly, it's **almost identical** to Gaussian elimination:

* You still perform the same elimination steps.
* Instead of modifying $A$ directly, you store the multipliers in $L$.

So the flop count is also:

$$
\approx \frac{n^3}{3}
$$

### ✅ Pros:

* Efficient if solving $Ax = b$ for **multiple** right-hand sides $b$, because once you have $LU$, you just do:

  * Forward substitution for $Ly = b$
  * Backward substitution for $Ux = y$
* Structured and reusable.

### ❌ Cons:

* Slightly more storage (you keep both $L$ and $U$).
* Without pivoting, numerical instability can occur.

---

## 💡 Example: 2x2 Case

For a 2x2 matrix:

$$
A = \begin{bmatrix} a & b \\ c & d \end{bmatrix}
$$

Only **one** elimination step is required:

* Multiply row 1 by $m = c/a$
* Subtract from row 2.

This step costs:

* 1 division (to compute the multiplier)
* 2 multiplications (apply to row)
* 2 subtractions (row update)

So \~5 flops.

Whether we’re using **Gaussian Elimination** or **LU**, this same step is done — the difference is **where we store the multiplier** (in $L$ or not).

---

## 🧾 Summary Table

| Method                         | Total Flops (approx) | Stores L separately | Best Use Case                          |
| ------------------------------ | -------------------- | ------------------- | -------------------------------------- |
| Gaussian Elimination           | $\frac{n^3}{3}$      | ❌ No                | Single system solve $Ax = b$           |
| LU Decomposition (no pivoting) | $\frac{n^3}{3}$      | ✅ Yes               | Multiple solves, matrix analysis, etc. |


Now let’s go deeper into the **cases when row exchanges are necessary**, how **permutation matrices** are used to represent those exchanges, and what their properties are.

---

## 🔄 Why Do We Need Row Exchanges?

In **Gaussian elimination** or **LU decomposition**, row exchanges are used when:

* The **pivot element is zero** (you can't divide by zero).
* The pivot is very **small**, which could lead to **numerical instability**.

### 🧠 Example:

$$
A = \begin{bmatrix}
0 & 2 & 3 \\
1 & 5 & 6 \\
4 & 7 & 9
\end{bmatrix}
$$

Here, the pivot in the (1,1) position is zero. We can’t proceed with elimination unless we **swap rows**, say row 1 and row 3.

---

## 🟦 What Is a Permutation Matrix?

A **permutation matrix** is a **square matrix** obtained by **reordering the rows** of the **identity matrix**. It represents a specific **row swap (or more generally, a row permutation)**.

### How it works:

If $P$ is a permutation matrix, then:

$$
PA = \text{Matrix A with rows permuted}
$$

### Properties:

* All entries are **0 or 1**.
* Each **row and column has exactly one 1**.
* $P^{-1} = P^T$ — because row swaps are **orthogonal operations**.
* Permutation matrices are **orthogonal**: $P^T = P^{-1}$

---

## 📌 Examples: 3×3 Permutation Matrices

### 1. Identity (no swap)

$$
P_0 = \begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
\Rightarrow \text{No change}
$$

### 2. Swap Row 1 and Row 2

$$
P_1 = \begin{bmatrix}
0 & 1 & 0 \\
1 & 0 & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

### 3. Swap Row 1 and Row 3

$$
P_2 = \begin{bmatrix}
0 & 0 & 1 \\
0 & 1 & 0 \\
1 & 0 & 0
\end{bmatrix}
$$

### 4. Swap Row 2 and Row 3

$$
P_3 = \begin{bmatrix}
1 & 0 & 0 \\
0 & 0 & 1 \\
0 & 1 & 0
\end{bmatrix}
$$

You can also have more complex permutations like rotating all rows:

### 5. Rotate rows: Row1 → Row2, Row2 → Row3, Row3 → Row1

$$
P_4 = \begin{bmatrix}
0 & 1 & 0 \\
0 & 0 & 1 \\
1 & 0 & 0
\end{bmatrix}
$$

---

## 🔁 Inverse of a Permutation Matrix

Since a permutation matrix **only shuffles rows**, its inverse is the matrix that **undoes that shuffle** — which is the **transpose**:

$$
P^{-1} = P^T
$$

This is because permutation matrices are **orthogonal** (like rotations and reflections in Euclidean space).

### ✅ Check with Example:

Let’s use $P_1$:

$$
P_1 = \begin{bmatrix}
0 & 1 & 0 \\
1 & 0 & 0 \\
0 & 0 & 1
\end{bmatrix}, \quad
P_1^T = \begin{bmatrix}
0 & 1 & 0 \\
1 & 0 & 0 \\
0 & 0 & 1
\end{bmatrix}
= P_1^{-1}
$$

$$
P_1 \cdot P_1^T = I
$$

---

## 🔁 How Is This Used in LU Decomposition?

When row swaps are needed during LU decomposition, we track them using a **permutation matrix $P$**, and the factorization becomes:

$$
PA = LU
$$

Where:

* $P$ is the product of all row swaps.
* $L$ is the lower triangular matrix with multipliers.
* $U$ is the upper triangular matrix.

This is called **LU decomposition with partial pivoting**.

---

## 🔚 Summary

| Concept            | Description                         |
| ------------------ | ----------------------------------- |
| Row exchange       | Needed when pivot is 0 or small     |
| Permutation matrix | Represents row swaps                |
| Entries            | 0s and 1s, one 1 per row and column |
| Inverse            | $P^{-1} = P^T$                      |
| Used in LU         | $PA = LU$ for stability             |


In [8]:
def lu_decomposition_with_pivoting(A):
    """
    Perform LU decomposition with partial pivoting.
    Returns permutation matrix P, lower triangular matrix L, and upper triangular matrix U
    such that PA = LU.
    """
    A = A.astype(float)
    n = A.shape[0]
    P = np.eye(n)
    L = np.zeros((n, n))
    U = A.copy()

    for i in range(n):
        # Find the pivot index
        pivot_index = np.argmax(np.abs(U[i:, i])) + i
        if U[pivot_index, i] == 0:
            raise ValueError("Matrix is singular!")

        # Swap rows in U and update P and L
        if pivot_index != i:
            U[[i, pivot_index]] = U[[pivot_index, i]]
            P[[i, pivot_index]] = P[[pivot_index, i]]
            if i > 0:
                L[[i, pivot_index], :i] = L[[pivot_index, i], :i]

        # Elimination
        for j in range(i+1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j] = U[j] - L[j, i] * U[i]

    # Fill diagonal of L with ones
    np.fill_diagonal(L, 1)

    return P, L, U

# Example matrix:
A = np.array([
    [0, 2, 3],
    [1, 5, 6],
    [4, 7, 9]
], dtype=float)

P, L, U = lu_decomposition_with_pivoting(A)

print("Original Matrix A:")
print(A)
print("\nPermutation Matrix P:")
print(P)
print("\nLower Triangular Matrix L:")
print(L)
print("\nUpper Triangular Matrix U:")
print(U)

# Verify PA = LU
print("\nCheck: PA:")
print(P @ A)
print("\nCheck: LU:")
print(L @ U)


Original Matrix A:
[[0. 2. 3.]
 [1. 5. 6.]
 [4. 7. 9.]]

Permutation Matrix P:
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]

Lower Triangular Matrix L:
[[1.         0.         0.        ]
 [0.25       1.         0.        ]
 [0.         0.61538462 1.        ]]

Upper Triangular Matrix U:
[[4.         7.         9.        ]
 [0.         3.25       3.75      ]
 [0.         0.         0.69230769]]

Check: PA:
[[4. 7. 9.]
 [1. 5. 6.]
 [0. 2. 3.]]

Check: LU:
[[4. 7. 9.]
 [1. 5. 6.]
 [0. 2. 3.]]
